# 1. Import

In [ ]:
# Basic Library
import pandas as pd
import pickle

# Preprocessing
from feature_engine import transformation as vt

# Modeling
import h2o
from h2o.automl import H2OAutoML

# 2. H2O

In [ ]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
submission = pd.read_csv("../data/sample_submission.csv")

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [ ]:
train['Water Weight'] = train['Whole Weight'] - (train['Shucked Weight'] + train['Shell Weight'])
test['Water Weight'] = test['Whole Weight'] - (test['Shucked Weight'] + test['Shell Weight'])

train.loc[train[(train['Water Weight']<0.0005)].index, "Water Weight"] = 0.0005
test.loc[test[(test['Water Weight']<0.0005)].index, "Water Weight"] = 0.0005

In [ ]:
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

In [ ]:
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

train = h2o.import_file("../data/train_ce")
test = h2o.import_file("../data/test_ce")

In [ ]:
x = train.columns
y = "Target"
x.remove(y)

In [ ]:
aml = H2OAutoML(max_models=20, stopping_metric="MAE", sort_metric="MAE", seed=1, project_name="H2O")
aml.train(x=x, y=y, training_frame=train)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
preds = aml.predict(test)